In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import np2d
import numpy as np
import pandas as pd
import networkx as nx

from latticeproteins.sequences import _residues
from latticeproteins.conformations import Conformations
from latticeproteins.thermodynamics import LatticeThermodynamics

In [3]:
def fixation(fitness1, fitness2, *args, **kwargs):
    """ Simple Gillespie fixation probability between two organism with fitnesses 1 and 2.
    (With infinite population size!)

    .. math::
        p_{\\text{fixation}} = \\frac{1 - e^{-N \\frac{f_2-f_1}{f1}}}{1 - e^{-\\frac{f_2-f_1}{f1}}}
    """
    sij = (fitness2 - fitness1)/abs(fitness1)
    # Check if any nans exist if an array of fitnesses is given.
    popsize = np.inf
    fixation = (1 - np.exp(-sij)) / (1 - np.exp(-popsize*sij))
    return  fixation

In [4]:
confs = Conformations(6)
lattice = LatticeThermodynamics(1.0, confs)

In [5]:
def build_M_fixation(anc, lattice):
    """Construct a transiton matrix for a given lattice sequence."""
    # Calculate 
    seq = list(anc)
    target = lattice.native_conf(anc)
    fitness_ref = lattice.fracfolded(seq, target=target)
    
    # Zip site number to site
    index = tuple(zip(range(len(anc)), anc))
    M_fitness = pd.DataFrame(index=index, columns=_residues, dtype=float)
    
    # Fit in fitnesses
    for AA in M_fitness.columns:
        for i, site in M_fitness[AA].index:
            seq2 = seq[:]
            seq2[i] = AA
            M_fitness[AA][(i,site)] = lattice.fracfolded(seq2, target=target)
            
    # Calculate a transition matrix
    M_fixation = fixation(fitness_ref, M_fitness)
    M_fixation.fillna(0, inplace=True)
    return M_fixation

In [190]:
def sample(anc, n_branch=3):
    """Sample evolutionary moves in sequence space that are a single mutation away.
    
    The two things we care about are 
    1. sequences
    2. fixation probability of that sequence (normalized over 3 moves)
    """
    # Fixation matrix
    M_fixation = build_M_fixation(anc, lattice)
    
    # normalize to get transition matrix
    M_transition = M_fixation / M_fixation.sum().sum()

    # Sample moves, if moves are not possible, break
    try:
        fixations, ilocs = np2d.random.choice(M_transition, size=n_branch, replace=False, p=M_transition)
        
        # Get new sequences
        edges = []
        for ith, iloc in enumerate(ilocs):
            site = M_fixation.index[iloc[0]][0]
            mutation = M_fixation.columns[iloc[1]]
            seq = list(anc)
            seq[site] = mutation
                        
            # Build an edge tuple
            tup = (anc, "".join(seq), fixations[ith] / fixations.sum())
            edges.append(tup)

        return edges

    # If there are not 3 possible moves, 
    except ValueError:
        return []

In [191]:
anc = "MACMAC"
n_moves = 4
n_branches = 3 

# Store each move in a dictionary
# Keys are the step number, values are dictionaries.
# these sub-dictionaries are edges { (anc, neighbor) : "fixation (normed)"}
# The fixation probabilities are normalized to 3 neighbors a particular
# egde is facing. 
moves = {i : [] for i in range(1, n_moves+1)}
moves[0] = [anc]
edges = []
neighbors = {}

# Walk n_moves.
for ith_move in range(1,n_moves+1):
    
    # Get current moves to explore
    current = moves[ith_move-1]

    for der in current:

        # Sample 3 moves
        edge = sample(der, n_branch=n_branches)

        future = [link[1] for link in edge]
        moves[ith_move] += future

        edges += edge

In [195]:
G = nx.DiGraph()
G.add_weighted_edges_from(edges)

In [196]:
trajectory_list = []
for target in G.nodes():
    trajectory_list += nx.all_simple_paths(G, anc, target)

In [198]:
# Get all trajectories
trajectory_list = []
for target in G.nodes():
    trajectory_list += nx.all_simple_paths(G, anc, target)
    
# Organize trajectories and calc their probability
trajectories = {i : {} for i in range(1,n_moves+1)}

for traj in trajectory_dict.values():
    # trajectory length
    length = len(traj)-1
    if length < 1:
        continue
    
    traj = tuple(traj)
    
    # trajectorie probability
    fix = 1
    for j in range(1, length+1):
        i = j - 1
        source, target = traj[i], traj[j]
        
        # Get total fixation out of node for normalization
        #norm = sum([G.edge[source][n]["weight"] for n in G.neighbors(source)])
        fix *= G.edge[source][target]["weight"]# / norm
        
        #fix *= moves[length][(source, target)]
        #edges[traj[i], traj[j]]
    
    try:
        trajectories[length][traj] += fix
    except KeyError:
        trajectories[length][traj] = fix

MACMAC NACMAC


KeyError: 'NACMAC'

In [177]:
sum(trajectories[2].values())

0.82492109604220909